In [ ]:
import numpy as np
import pandas as pd
from scripts.python.routines.manifest import get_manifest
from scripts.python.preprocessing.serialization.routines.pheno_betas_checking import get_pheno_betas_with_common_subjects
import pathlib
from tqdm import tqdm
from impyute.imputation.cs import fast_knn, mean, median, random, mice, mode, em


path = f"E:/YandexDisk/Work/pydnameth/datasets/meta/tasks/GPL13534_Blood/"
manifest = get_manifest('GPL13534')

disease = "Parkinson" # "Parkinson" "Schizophrenia"
data_type = "non_harmonized"

datasets = ["data_trn_val", "data_tst_GSE72774"]

path_load = f"{path}/{disease}/{data_type}"

df_trn_val = pd.read_pickle(f"{path_load}/data_trn_val.pkl")
df_tst = pd.read_pickle(f"{path_load}/data_tst_GSE72774.pkl")

In [ ]:
common_columns = df_trn_val.columns.intersection(df_tst.columns).values

In [ ]:
df_all = pd.concat([df_trn_val.loc[:, common_columns], df_tst.loc[:, common_columns]])

In [ ]:
cpgs_all = list(set(manifest.index.values).intersection(set(df_all.columns.values)))

In [ ]:
df_all.to_pickle(f"{path_load}/data_trn_val_tst_GSE72774.pkl")

In [11]:
df_cpgs_all = pd.DataFrame(index=cpgs_all)
df_cpgs_all.to_excel(f"{path_load}/cpgs/{len(cpgs_all)}_trn_val_tst_GSE72774.xlsx", index=True, index_label="features")

In [13]:
df_cpgs_selected = pd.read_excel(f"{path_load}/cpgs/43019.xlsx", index_col='features')
df_cpgs_selected = df_cpgs_selected.loc[df_cpgs_selected.index.isin(cpgs_all), :]
df_cpgs_selected.to_excel(f"{path_load}/cpgs/{df_cpgs_selected.shape[0]}_trn_val_tst_GSE72774.xlsx", index=True, index_label="features")